In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import tree
from sklearn import svm
from sklearn import neighbors
from sklearn import linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
data = pd.read_csv("historical_data.csv")

In [ ]:
# GOAL : TO PREDICT THE DELIVERY DURATION, I.E., DIFFERENCE BETWEEN CREATED_AT AND ACTUAL_DELIVERY_TIME
data.head()

In [ ]:
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])

In [ ]:
data['delivery_duration'] = (data['actual_delivery_time'] - data['created_at'])

In [ ]:
data['delivery_duration'] = (data['delivery_duration']).dt.total_seconds()

In [ ]:
data['busy_dashers_ratio'] = (data['total_busy_dashers']/data['total_onshift_dashers'])

In [ ]:
data['estimated_non_prep_duration'] = data['estimated_order_place_duration'] + data['estimated_store_to_consumer_driving_duration']

In [ ]:
data['market_id'].nunique()

In [ ]:
# if we use this, we'll have too many dummies, which will explode the dataset
data['store_id'].nunique()

In [ ]:
# good size for dummies
# order protocol is the mode in which doordash receives orders.
data['order_protocol'].nunique()

In [ ]:
market_id_dummies = pd.get_dummies(data.market_id)
market_id_dummies = market_id_dummies.add_prefix('market_id_')

order_protocol_dummies = pd.get_dummies(data.order_protocol)
order_protocol_dummies = order_protocol_dummies.add_prefix('order_protocol_')

In [ ]:
market_id_dummies.head()

In [ ]:
store_id_unique = data['store_id'].unique().tolist()
store_id_and_category = {store_id : data[data.store_id == store_id].store_primary_category.mode() for store_id
                        in store_id_unique}

In [ ]:
def fill(store_id):
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan 

In [ ]:
data['store_category_without_null'] = data['store_id'].apply(fill)

In [ ]:
# dummies for store_category
store_category_dummies = pd.get_dummies(data.store_category_without_null)
store_category_dummies = store_category_dummies.add_prefix('category_')

In [ ]:
# dropping a few columns, where we replace them with their dummy columns. 

train_data = data.drop(columns = ['market_id', 'created_at', 'actual_delivery_time', 'store_id',
                                 'store_primary_category', 'store_category_without_null', 'order_protocol'])

In [ ]:
# concatenating the dummy columns to our training data
train_data = pd.concat([train_data, market_id_dummies, store_category_dummies, order_protocol_dummies], axis = 1)

In [ ]:
# change dtype to float for modelling
train_data = train_data.astype('float32')

In [ ]:
train_data

In [ ]:
train_data['total_busy_dashers'].describe()

In [ ]:
# calculating the correlation matrix
corr = train_data.corr()

In [ ]:
corr

In [ ]:
fig, ax = plt.subplots(figsize = (25,25)) # size of the graph 25x25 is perfect
cmap = sns.diverging_palette(230, 20, as_cmap = True)
sns.heatmap(corr, cmap = cmap, vmax = .3, linewidths = 0.5, center = 0, square = True, cbar_kws = {'shrink':.5})

In [ ]:
train_data.shape[1]

In [ ]:
def get_redundant_pairs(df):
    redundant_pairs = set()
    cols = df.columns
    for i in range(0,df.shape[1]):
        for j in range(0,i+1):
            redundant_pairs.add((cols[i],cols[j]))
    return redundant_pairs

In [ ]:
def top_abs_corrs(df, n = 5):
    au_corr = df.corr().abs().unstack()
    # series object that contains all the pairwise correlations of the columns given.
    items_to_drop = get_redundant_pairs(df) #find out the items from the upper triangle and diagonal
    au_corr = au_corr.drop(labels = items_to_drop).sort_values(ascending = False)
    # drops the excess items (5050 in case of train_data) to make processing faster.
    return au_corr[0:n] #returns the n-highest correlations

In [ ]:
top_abs_corrs(train_data, 20)

In [ ]:
# we'll drop the highly correlated features
train_data = train_data.drop(columns = ['total_onshift_dashers', 'total_busy_dashers',
                                       'estimated_non_prep_duration', ])

In [ ]:
train_data = train_data.drop(order_protocol_dummies, axis = 1) # dropping the set of all order_protocol_dummies

In [ ]:
train_data = train_data.drop(market_id_dummies, axis = 1) # dropping the set of all market_id_dummies

In [ ]:
top_abs_corrs(train_data, 20)

In [ ]:
# engineering new features and dropping the ones previously
train_data['avg_item_price'] = train_data['subtotal']/train_data['total_items']
train_data['order_price_range'] = train_data['max_item_price'] - train_data['min_item_price']
train_data['distinct_item_percent_of_total'] = train_data['num_distinct_items']/train_data['total_items']

In [ ]:
train_data = train_data.drop(columns = ['total_items', 'subtotal', 'max_item_price', 'min_item_price',
                                       'num_distinct_items'])

In [ ]:
train_data['category_indonesian'].describe()

In [ ]:
train_data = train_data.drop(columns = ['category_indonesian'])

In [ ]:
top_abs_corrs(train_data, 20)

In [ ]:
train_data = train_data.astype('float32') #converting to suitable datatype
# converting infinity values to null values to finally drop them
train_data.replace([np.inf, -np.inf], np.nan, inplace = True)
train_data.dropna(inplace = True)

# we have significantly reduced the collinearity problem so now we can focus on something else.

In [ ]:
# we'll now try to reduce the multicollinearity problem and work on that using VIF and PCA. we'll also use certain
# regression techniques to find the importance (gini purity of each feature) to reduce the dimensions of the dataset.

In [ ]:
# VIF(variance inflation factor) - basically tests for how good a variable can be approximated based on a linear
# combination of all other variables. it is basically like finding the R^2 of the regression having each of the 
# dependent variables regressed on all the other dependent variables. VIF = 1/1-R^2i, where R^2i is the score for the
# regression of the i th dependent variable on all other dependent variables.


In [ ]:
def compute_vif(features):
    vif_data = pd.DataFrame()
    vif_data['features'] = features
    vif_data['VIF'] = [variance_inflation_factor(train_data[features].values, i ) for i in range(len(features))]
    return vif_data.sort_values(by = ['VIF']).reset_index(drop = True)

In [ ]:
# converting features to a list
features = train_data.drop(columns = ['delivery_duration']).columns.to_list()

In [ ]:
vif_data = compute_vif(features)

In [ ]:
# function to remove features with vif score higher than a certain upper bound
multicoll = True

while multicoll:
    highest_vif_feature = vif_data['features'].values.tolist()[-1]
    features.remove(highest_vif_feature)
    vif_data = compute_vif(features)
    if len(vif_data[vif_data['VIF'] > 15]) == 0 :
        multicoll = False
    else:
        multicoll = True

vif_data

# we now have effectively 79 columns. the next step is to see the importance of each of these features in predicting
# the target variable.

In [ ]:
# we will reduce the dimensions further by using PCA and random forests regression to find the most important features
# by predicitve capability.

In [ ]:
selected_features = vif_data['features'].values.tolist()

In [ ]:
train_data[selected_features]

In [ ]:
x = train_data[selected_features] #reduced feature set i.e. with 79 features
y = train_data['delivery_duration'] #target variable

In [ ]:
x_train, x_test, y_train, y_test, = train_test_split(x, y, test_size = 0.2, random_state = 42) 

In [ ]:
feature_set = [f'feature {i}' for i in range(x.shape[1])]

In [ ]:
forest = RandomForestRegressor(random_state = 42)
forest.fit(x_train, y_train)

In [ ]:
feats = {} # a dictionary that holds the feature and their gini importance pairs
for feature, importance in zip(x.columns, forest.feature_importances_): # zip pairs up the items in the two iterables listed
    feats[feature] = importance #adds the name-value pair

importances = pd.DataFrame

In [ ]:
importances = importances.from_dict(feats, orient = 'index') # takes the data from the dictionary

In [ ]:
importances = importances.rename(columns = {0 : 'gini_importance'}) # renames the column to gini importance

In [ ]:
importances = importances.sort_values(by = 'gini_importance') # sorts values in ascending order

In [ ]:
importances.plot(kind = 'bar', rot = 90, figsize = (25,25)) # plots the dataframe
plt.show()

In [ ]:
# we use PCA to further the dimensionality reduction process. we use 2 types of scalers - minmax and standard.
# the problem with these scaling techniques is their high sensitivity to outliers when compared to more robust scalers
# like the robustscaler. nevertheless, we continue with the project using this

x_train = x_train.values
x_train = np.asarray(x_train) # not sure why we need to do this since our data is already in ndarray format

x_std = StandardScaler().fit_transform(x_train) # fit trains our ML model. read more about this in your next project

pca = PCA().fit(x_std) # gives the PCA of the data fed

# visualizing the PCA insights
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,81,1) # sets the limits on the x-axis
plt.xlabel('number of features') # labels the x-axis
plt.ylabel('cumulative explained variance') # labels the y-axis
plt.show()

In [ ]:
def scale(scaler_type, x, y):
    scaler = scaler_type
    scaler.fit(x) # not sure why we need to specify the scaler type here if the scaling (i.e. transforming
    # isnt actually happening. read on this later.)
    x_scaled = scaler.transform(x)
    scaler.fit(y.values.reshape(-1,1)) #understand why we need to do the reshaping
    y_scaled = scaler.transform(y.values.reshape(-1,1))
    
    return (x_scaled, y_scaled, scaler, scaler)


In [ ]:
# applying the scaling function
x_scaled, y_scaled, x_scaler, y_scaler = scale(MinMaxScaler(), x, y)
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(x_scaled, y_scaled, test_size = 0.2,
                                                                               random_state = 42)
# the randomization gives us the same random entry sets as before since we mentioned the same random_state i.e. 42.

In [ ]:
# function for scaling the errors back to their pre-transformation units
from sklearn.metrics import mean_squared_error

def rmse_with_inv_transform(scaler, y_test, y_pred_scaled, model_name):
    y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1,1))
    rmse_error = mean_squared_error(y_test.values.reshape(-1,1)[: , 0], y_pred[: , 0], squared = False)
    print('Error = {}'.format(rmse_error) + ' in ' + model_name)
    return rmse_error, y_predict

In [ ]:
def make_regression(x_train, y_train, x_test, y_test, model, model_name, verbose = False):
    model.fit(x_train, y_train) # trains the data according to the model
    y_predict = model.predict(x_train) # gives the predictions on the fitted data
    train_error = mean_squared_error(y_train, y_predict, squared = False) #rmse for training
    y_predict = model.predict(x_test) # gives the predictions on the test data which is not fitted (not sure why 
    # we arent fitting the test data but let's see that later.)
    test_error = mean_squared_error(y_test, y_predict, squared = False) #rmse for testing
    if verbose:
        print('Train_error = {}'.format(train_error) + ' in ' + model_name)
        print('Test_error = {}'.format(test_error) + ' in ' + model_name)
    
    trained_model = model
    
    return trained_model, y_predict, train_error, test_error

In [ ]:
pred_dict = {
    'regression_model' : [],
    'feature_set' : [],
    'scaler' : [],
    'RMSE' : []
}

regression_models = {
    'Ridge' : linear_model.Ridge(),
    'DecisionTree' : tree.DecisionTreeRegressor(max_depth = 6),
    'RandomForest' : RandomForestRegressor(),
    'XGBoost' : XGBRegressor(),
    'MLP' : MLPRegressor()
}

feature_sets = {
    'full_dataset' : importances.index.to_list(),
    'top_40_features' : importances.sort_values(by = 'gini_importance')[-40: ].index.to_list(),
    'top_20_features' : importances.sort_values(by = 'gini_importance')[-20: ].index.to_list(),
    'top_10_features' : importances.sort_values(by = 'gini_importance')[-10: ].index.to_list()    
}

scalers = {
    'standard_scaler' : StandardScaler(),
    'MinMax_scaler' : MinMaxScaler(),
    'Robust_scaler' : RobustScaler(),
    'No_scale' : None
    
}

In [ ]:
for feature_set_name in feature_sets.keys():
    feature_set = feature_sets[feature_set_name] # returns the set of features accordingly
    for scaler_name in scalers.keys():
        print(f'scaled with -------{scaler_name}-------using {feature_set_name}')
        print('')
        for model_name in regression_models.keys():
            x = train_data[feature_set]
            y = train_data['delivery_duration']
            if scaler_name == 'No_scale':
                x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
                make_regression(x_train, y_train, x_test, y_test, regression_models[model_name], model_name)
            else:
                x_scaled, y_scaled, x_scaler, y_scaler = scale(scalers[scaler_name], x, y)
                x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
                    x_scaled, y_scaled, test_size = 0.2, random_state = 42)
                _, y_predict_scaled, _, _ = make_regression(x_train_scaled, y_train_scaled[:, 0], x_test_scaled,
                                                           y_test_scaled, regression_models[model_name], model_name)
                rmse_error, y_predict = rmse_with_inv_transform(y_scaler, y_test, y_predict_scaled, model_name)
            pred_dict['regression_model'].append(model_name)
            pred_dict['feature_set'].append(feature_set_name)
            pred_dict['scaler'].append(scaler_name)
            pred_dict['RMSE'].append(rmse_error)

In [ ]:
predictions_data = pd.DataFrame(pred_dict)

In [ ]:
predictions_data

In [ ]:
predictions_data.plot(kind = 'bar', figsize = (15,15))

In [ ]:
train_data

In [ ]:
# changing the problem and defining a new variable and estimating that
train_data['prep_time'] = train_data['delivery_duration'] - train_data['estimated_order_place_duration'] - train_data['estimated_store_to_consumer_driving_duration']


In [ ]:
# since the variation in scaling and feature_set sizes doesn't change the output by much, we can just use the most
# convenient options. we will use robust scaler and feature_set_of_40.

scaler = RobustScaler()
feature_set = importances.sort_values(by = 'gini_importance')[-40:].index.to_list()


In [ ]:
# implementing the model with the new features on the new variable (i.e. prep_time)

print(f'scaled with -------{scaler}-------using top 40 features')
print('')
for model_name in regression_models.keys():
    x = train_data[feature_set].drop(columns = ['estimated_order_place_duration',
                                                'estimated_store_to_consumer_driving_duration'])
    y = train_data['prep_time']
    
    x_scaled, y_scaled, x_scaler, y_scaler = scale(scaler, x, y)
    x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    x_scaled, y_scaled, test_size = 0.2, random_state = 42)
    _, y_predict_scaled, _, _ = make_regression(x_train_scaled, y_train_scaled[:, 0], x_test_scaled,
                                                           y_test_scaled, regression_models[model_name], model_name)
    rmse_error, y_predict = rmse_with_inv_transform(y_scaler, y_test, y_predict_scaled, model_name)

In [ ]:
regression_models = {'XGBoost' : XGBRegressor()}

print(f'scaled with -------{scaler}-------using top 40 features')
print('')
for model_name in regression_models.keys():
    x = train_data[feature_set].drop(columns = ['estimated_order_place_duration',
                                                    'estimated_store_to_consumer_driving_duration'])
    y = train_data['prep_time']
    
    x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 42)
    
    # get indices of the rows in training and testing datasets
    
    train_indices = x_train.index
    test_indices = x_test.index
    
    
    x_scaled, y_scaled, x_scaler, y_scaler = scale(scaler, x, y)
    x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    x_scaled, y_scaled, test_size = 0.2, random_state = 42)
    _, y_predict_scaled, _, _ = make_regression(x_train_scaled, y_train_scaled[:, 0], x_test_scaled,
                                                               y_test_scaled, regression_models[model_name], model_name)
    rmse_error, y_predict = rmse_with_inv_transform(y_scaler, y_test, y_predict_scaled, model_name)

In [ ]:
# storing the predicted values in a dictionary
predicted_values = {
    'total_delivery_duration' : train_data['delivery_duration'][test_indices].values.tolist(),
    'prep_duration_prediction' : y_predict.tolist(),
    'estimated_order_place_duration' : train_data['estimated_order_place_duration'][test_indices].values.tolist(),
    'estimated_store_to_consumer_driving_duration' : train_data['estimated_store_to_consumer_driving_duration'][test_indices].values.tolist()
}

In [ ]:
# CONVERTING THE DICTIONARY TO A DATAFRAME AND DEFINING TH CONCERNED VARIABLE AGAIN
values_data = pd.DataFrame.from_dict(predicted_values)
values_data
values_data['estimated_total_delivery_duration'] = values_data['prep_duration_prediction'] + values_data['estimated_order_place_duration'] + values_data['estimated_store_to_consumer_driving_duration']
mean_squared_error(values_data['total_delivery_duration'], values_data['estimated_total_delivery_duration'], squared = False)

In [ ]:
# RUNNING A NEW REGRESSION WITH ONLY THE MOST RELEVANT FEATURES

print(f'scaled with -------{scaler}-------using top 3 features')
print('')
for model_name in regression_models.keys():
    x = values_data[['estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration', 'prep_duration_prediction']]
    y = values_data['total_delivery_duration']
    
    x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 42)
    
    x_scaled, y_scaled, x_scaler, y_scaler = scale(scaler, x, y)
    x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    x_scaled, y_scaled, test_size = 0.2, random_state = 42)
    _, y_predict_scaled, _, _ = make_regression(x_train_scaled, y_train_scaled[:, 0], x_test_scaled,
                                                           y_test_scaled, regression_models[model_name], model_name)
    rmse_error, y_predict = rmse_with_inv_transform(y_scaler, y_test, y_predict_scaled, model_name)